## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "Green Taxi Trip Records", we'll use "For-Hire Vehicle Trip Records" 

Download the data for January and February 2021

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

In [ ]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet

In [ ]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet

Read the data for January. How many records are there?

In [1]:
import pandas as pd

In [40]:
df = pd.read_parquet('fhv_tripdata_2021-01.parquet')
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,NaN,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,NaN,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,NaN,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,NaN,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,NaN,B00037
...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,NaN,B03266
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,NaN,
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,NaN,B03285
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,NaN,B03285


>Answer: 1154112

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the average trip duration in January?

In [41]:
df[['pickup_datetime', 'dropOff_datetime']]

,pickup_datetime,dropOff_datetime
0,2021-01-01 00:27:00,2021-01-01 00:44:00
1,2021-01-01 00:50:00,2021-01-01 01:07:00
2,2021-01-01 00:01:00,2021-01-01 01:51:00
3,2021-01-01 00:13:09,2021-01-01 00:21:26
4,2021-01-01 00:38:31,2021-01-01 00:53:44
...,...,...
1154107,2021-01-31 23:43:03,2021-01-31 23:51:48
1154108,2021-01-31 23:50:27,2021-02-01 00:48:03
1154109,2021-01-31 23:13:46,2021-01-31 23:29:58
1154110,2021-01-31 23:58:03,2021-02-01 00:17:29


In [42]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [43]:
df.duration.mean()

19.167224093791006

>Answer: 19.16

## Data preparation

Check the distribution of the duration variable. There are some outliners. 

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [44]:
df.describe()

,PUlocationID,DOlocationID,SR_Flag,duration
count,195845.000000,991892.000000,0.0,1.154112e+06
mean,139.859690,135.898030,NaN,1.916722e+01
std,74.991382,80.474902,NaN,3.986922e+02
min,1.000000,1.000000,NaN,1.666667e-02
25%,75.000000,67.000000,NaN,7.766667e+00
50%,143.000000,132.000000,NaN,1.340000e+01
75%,206.000000,213.000000,NaN,2.228333e+01
max,265.000000,265.000000,NaN,4.233710e+05


In [45]:
df.count()

dispatching_base_num      1154112
pickup_datetime           1154112
dropOff_datetime          1154112
PUlocationID               195845
DOlocationID               991892
SR_Flag                         0
Affiliated_base_number    1153227
duration                  1154112
dtype: int64

In [46]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [47]:
df.describe()

,PUlocationID,DOlocationID,SR_Flag,duration
count,182818.000000,961919.000000,0.0,1.109826e+06
mean,139.861633,135.689147,NaN,1.624725e+01
std,74.763788,80.349678,NaN,1.155150e+01
min,1.000000,1.000000,NaN,1.000000e+00
25%,75.000000,67.000000,NaN,7.850000e+00
50%,146.000000,132.000000,NaN,1.323333e+01
75%,206.000000,213.000000,NaN,2.146667e+01
max,265.000000,265.000000,NaN,6.000000e+01


In [48]:
df.count()

dispatching_base_num      1109826
pickup_datetime           1109826
dropOff_datetime          1109826
PUlocationID               182818
DOlocationID               961919
SR_Flag                         0
Affiliated_base_number    1109053
duration                  1109826
dtype: int64

How many records did you drop? 

In [22]:
1154112 - 1109826

44286

>Answer: 44286

## Q3. Missing values

The features we'll user for our model are the pickup and dropoff location IDs. 

But they have a lot of missing values there. Let's replace them with "-1"

In [51]:
df.isnull().sum()

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID               927008
DOlocationID               147907
SR_Flag                   1109826
Affiliated_base_number        773
duration                        0
dtype: int64

In [52]:
df['PUlocationID'] = df['PUlocationID'].fillna(-1)
df['DOlocationID'] = df['DOlocationID'].fillna(-1)

/var/folders/z6/wchwkzy93yqb85l4qc39505r0000gn/T/ipykernel_9753/1051174452.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PUlocationID'] = df['PUlocationID'].fillna(-1)
/var/folders/z6/wchwkzy93yqb85l4qc39505r0000gn/T/ipykernel_9753/1051174452.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOlocationID'] = df['DOlocationID'].fillna(-1)


In [53]:
df.isnull().sum()

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID                    0
DOlocationID                    0
SR_Flag                   1109826
Affiliated_base_number        773
duration                        0
dtype: int64

What's the factions of missing values for the pickup location ID? (Or the fraction of "-1"s after you filled the NAs)

In [55]:
total = 1109826
pu = 927008

print(pu/total*100)

83.52732770722618


>Answer: 83%

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer 
* Get a feature matrix from it

In [58]:
from sklearn.feature_extraction import DictVectorizer

In [57]:
categorical = ['PUlocationID', 'DOlocationID']
numerical = ['duration']

In [59]:
df[categorical] = df[categorical].astype(str)

/var/folders/z6/wchwkzy93yqb85l4qc39505r0000gn/T/ipykernel_9753/1767203464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [60]:
train_dicts = df[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

What's the dimensionality of this matrix? (The number of columns)

In [61]:
X_train

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

>Answer: 525

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

In [62]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [63]:
target = 'duration'
y_train = df[target].values

In [64]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519432127307

What's the RMSE on train?

>Answer: 10.52

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (Feb 2021). 

What's the RMSE on validation?

In [75]:
df_val = pd.read_parquet('fhv_tripdata_2021-02.parquet')
df_val['duration'] = df_val.dropOff_datetime - df_val.pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]
df_val[categorical] = df_val[categorical].astype(str)

In [76]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [77]:
y_val = df_val[target].values

In [78]:
y_pred = lr.predict(X_val)

In [79]:
mean_squared_error(y_val, y_pred, squared=False)

11.36407715557367

>Answer: 11.01